In [1]:
import pandas as pd
import numpy as np
from random import randint
from scipy import stats
import io

In [2]:
from google.colab import files
  
  
uploaded = files.upload()

Saving control_ab.csv to control_ab.csv


In [3]:
control = pd.read_csv(io.BytesIO(uploaded['control_ab.csv']))
control.head()

,Campaign Name,Date,Spend [USD],# of Impressions,Reach,# of Website Clicks
0,Control Campaign,1.08.2019,2280,82702,56930,7016
1,Control Campaign,2.08.2019,1757,121040,102513,8110
2,Control Campaign,3.08.2019,2343,131711,110862,6508
3,Control Campaign,4.08.2019,1940,72878,61235,3065
4,Control Campaign,6.08.2019,3083,109076,87998,4028


In [4]:
from google.colab import files
  
  
uploaded = files.upload()

Saving test_ab.csv to test_ab.csv


In [5]:
experiment = pd.read_csv(io.BytesIO(uploaded['test_ab.csv']))
experiment.head()

,Campaign Name,Date,Spend [USD],# of Impressions,Reach,# of Website Clicks
0,Test Campaign,1.08.2019,3008,39550,35820,3038
1,Test Campaign,2.08.2019,2542,100719,91236,4657
2,Test Campaign,3.08.2019,2365,70263,45198,7885
3,Test Campaign,4.08.2019,2710,78451,25937,4216
4,Test Campaign,5.08.2019,2297,114295,95138,5863


'# of Impressions' = Jumlah user yang melihat iklan
<br> Reach = Jumlah unique id (user) yang melihat iklan
<br> '# of Website Clicks' = Jumlah user yang mengklik iklan

Hipotesis:
<br>
<br> H0 : μ(eks) - μ(con) <= 0
<br> H1 : μ(eks) - μ(con) > 0

<br> Gunakan taraf nyata (α) = 0.5 (5%) dengan menggunakan z-test dikarenakan jumlah sampel yang digunakan berjumlah lebih dari 30

In [6]:
def var_ratio(x,y): #x/y
    mean_x = np.mean(x)
    mean_y = np.mean(y)
    var_x = np.var(x,ddof=1)
    var_y = np.var(y,ddof=1)
    cov_xy = np.cov(x,y,ddof=1)[0][1]
    result = (var_x/mean_x**2 + var_y/mean_y**2 - 2*cov_xy/(mean_x*mean_y))*(mean_x*mean_x)/(mean_y*mean_y*len(x))
    return result

In [7]:
def ttest(mean_control,mean_experiment,var_control,var_experiment):
    diff = mean_experiment - mean_control
    var = var_control+var_experiment
    stde = 1.96*np.sqrt(var)
    lower = diff - stde 
    upper = diff + stde
    z = diff/np.sqrt(var)
    p_val = stats.norm.sf(abs(z))*2

    result = {'mean_control':mean_control,
             'mean_experiment':mean_experiment,
             'var_control':var_control,
             'var_experiment':var_experiment,
             'difference':diff,
             'lower_bound':lower,
             'upper_bound':upper,
             'z_value':z,
             'p-value':p_val}
    return pd.DataFrame(result,index=[0])

In [8]:
var_control = var_ratio(control['# of Website Clicks'],control['# of Impressions'])
var_experiment = var_ratio(experiment['# of Website Clicks'],experiment['# of Impressions'])
mean_control = control['# of Website Clicks'].sum()/control['# of Impressions'].sum()
mean_experiment = experiment['# of Website Clicks'].sum()/experiment['# of Impressions'].sum()


ttest(mean_control,mean_experiment,var_control,var_experiment)

,mean_control,mean_experiment,var_control,var_experiment,difference,lower_bound,upper_bound,z_value,p-value
0,0.048565,0.080879,0.000013,0.000065,0.032314,0.014973,0.049654,3.652424,0.00026


Dengan α = 0.5 (5%) maka didapatkan z(1-α) = 1.645 yang berarti z(α) < z(value).

Sehingga tolak H0 dan terima H1. Keputusan ini juga diperkuat dengan nilai p-value yang bernilai < 0.01 yang berarti adanya bukti kuat untuk menolak H0. Jadi dapat disimpulkan bahwa tampilan iklan baru pada halaman website memberikan dampak terhadap perilaku pengunjung website untuk mengklik iklan tersebut